In [1]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import minimize
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [2]:
df_etf = pd.read_csv('data_etf.csv')
df_etf = df_etf.set_index('Date')
df_etf.index = pd.to_datetime(df_etf.index)
df_etf[df_etf.columns] = df_etf[df_etf.columns].astype(float)
df_etf = df_etf.sort_values(by = 'Date')

df_goodan = pd.read_csv('data_goodan.csv')
df_goodan = df_goodan.set_index('Date')
df_goodan.index = pd.to_datetime(df_goodan.index)
df_goodan[df_goodan.columns] = df_goodan[df_goodan.columns].astype(float)
df_goodan = df_goodan.sort_values(by = 'Date')

In [3]:
df_goodan[df_goodan.columns] = df_goodan[df_goodan.columns].astype(float)
df_etf[df_etf.columns] = df_etf[df_etf.columns].astype(float)

In [4]:
goodan = 'Ajax'

In [5]:
def goodan_selection(goodan) :
    
    df_data = df_etf.copy()
    df_data[goodan] = df_goodan[goodan]
    df_result = df_data.sort_values('Date')
    
    return df_result

In [6]:
data = goodan_selection(goodan)

In [7]:
asset_num = len(data.columns)

In [8]:
value = 1

In [15]:
ratio = 0.1

In [10]:
def daily_to_month(start, end, df_daily):
    
    s = pd.date_range(start, end, freq = 'MS')
    e = pd.date_range(start, end, freq = 'M')

    v = []

    for i, j in zip(s, e) :
        
        date = pd.date_range(i, j)
        
        df_date_range_value = df_daily.loc[date].dropna()
        
        value_result = df_date_range_value.iloc[[0]]
        
        v.append(value_result)

    df_month = pd.concat(v)
    
    return df_month

In [11]:
def price_to_return_rate(data) :
    
    df_2 = data
    
    df_pro = (df_2 - df_2.shift(1)).iloc[1:,:]
    df_pro2 = df_2.shift(1).iloc[1:,:]
    
    df_return = df_pro / df_pro2
    
    data = df_return.iloc[:,:]
    
    return data

In [12]:
df_month = daily_to_month('2014-01-01', '2019-01-01', data)
data = price_to_return_rate(df_month)
data.head()

,KOSEF 단기자금(A130730),KODEX 인버스(A114800),KODEX 은선물(H)(A144600),KODEX 삼성그룹(A102780),KODEX 레버리지(A122630),KODEX 골드선물(H)(A132030),KODEX 건설(A117700),KODEX 200(A069500),KINDEX 레버리지(A152500),KINDEX 200(A105190),Ajax
2014-02-03,0.001894,0.031291,-0.042198,-0.037829,-0.060696,0.016437,-0.004274,-0.027403,-0.062430,-0.026913,-0.007025
2014-03-03,0.002288,-0.020228,0.114754,0.010256,0.041121,0.085550,0.065808,0.022817,0.041617,0.021928,0.089777
2014-04-01,0.001985,-0.009032,-0.076287,0.012690,0.016059,-0.043248,-0.017450,0.009117,0.015982,0.008892,0.010880
2014-05-02,0.002328,0.017578,-0.038806,-0.020050,-0.029902,-0.000502,0.015027,-0.022684,-0.031461,-0.025867,0.021527
2014-06-02,0.001878,-0.023672,-0.015528,0.045183,0.046235,-0.031156,-0.030956,0.025177,0.047564,0.025964,-0.021073


In [52]:
def goodan_optimizer(value, asset_num, data, ratio) :

    data_etf = data
    r_cov = data_etf.cov()
    
    def objective(weights):
        return 0.5*np.sqrt(np.dot(weights.T, np.dot(r_cov, weights)))
    
    def constraint1(weights):
        return sum(weights)-1
    
    def constraint2(weights):
        return weights[-1] - ratio
    
    w0 = [(1 / asset_num) for i in range(asset_num)]
    w0 = np.array(w0)
    
    bound = [0,1]
    bnds = [bound for i in range(asset_num)]
    con1 = {'type':'eq','fun':constraint1}
    con2 = {'type':'eq','fun':constraint2}
    cons = [con1, con2]
    
    sol = minimize(objective,w0,method='SLSQP',bounds = bnds, constraints = cons)
    
    return value * sol.x.round(3)